In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
                    .appName("test") \
                    .master("local[4]") \
                    .config("spark.jars.packages", "mysql:mysql-connector-java:8.0.13,org.apache.spark:spark-hive_2.12:3.2.1") \
                    .config("spark.sql.warehouse.dir", "/opt/hive/data/warehouse") \
                    .config("hive.metastore.uris", "thrift://localhost:9083") \
                    .config("spark.sql.parquet.int96RebaseModeInWrite", "LEGACY") \
                    .config("spark.executor.memory", "8g") \
                    .config("spark.driver.memory", "4g") \
                    .config("spark.executor.instances", "4") \
                    .config("spark.executor.cores", "4") \
                    .config("spark.driver.cores", "2") \
                    .enableHiveSupport() \
                    .getOrCreate()

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy import MetaData

MYSQL_HOST = "localhost"
MYSQL_USER = "root"
MYSQL_PASSWORD = "Thao0611*"
MYSQL_DB = "dacn"

SQLALCHEMY_DATABASE_URL = f"mysql+mysqlconnector://{MYSQL_USER}:{MYSQL_PASSWORD}@{MYSQL_HOST}/{MYSQL_DB}"
# Create a SQLAlchemy engine
engine = create_engine(SQLALCHEMY_DATABASE_URL)
m = MetaData()
m.reflect(engine)
tables = []
for table in m.tables.values():
    tables.append(table.name)
print(tables)


In [ ]:
exist_tables = ['do_d_items']

In [ ]:
for table in tables:
    if table in exist_tables:
        continue
    print(table)
    df = spark.read \
        .format("jdbc") \
        .option("driver","com.mysql.cj.jdbc.Driver") \
        .option("url", "jdbc:mysql://localhost:3306/dacn?useSSL=false") \
        .option("dbtable", f"{table}") \
        .option("user", "root") \
        .option("password", "Thao0611*") \
        .load()
    spark.sql("CREATE DATABASE IF NOT EXISTS mimic_iii")
    df.write.mode("overwrite").saveAsTable(f"mimic_iii.{table}")
    

In [ ]:
spark.stop()

In [2]:
import numpy as np

spark.sql("DROP TABLE IF EXISTS mimic_iii.PROCEDUREEVENTS_MV")

df = spark.read \
        .format("jdbc") \
        .option("driver","com.mysql.cj.jdbc.Driver") \
        .option("url", "jdbc:mysql://localhost:3306/dacn?useSSL=false") \
        .option("dbtable", "PROCEDUREEVENTS_MV") \
        .option("user", "root") \
        .option("password", "Thao0611*") \
        .load()

df.write.mode("append").saveAsTable(f"mimic_iii.PROCEDUREEVENTS_MV")

print("Read successfully!")


Read successfully!


In [3]:
df.repartition("hadm_id").write.partitionBy("hadm_id").mode("append").saveAsTable("mimic_iii.do_d_items")
